In [ ]:
# Import libraries
import glob, os
import pandas as pd
import numpy as np

In [ ]:
# Set directory and dataset names.
base_dir = "/mnt/mbServerData/data/AOMIC"
datasets = ["ID1000"]

In [ ]:
sub_motion = []

# Take mean framewise displacement and DVARS from each confound file.
for dataset in datasets: 
    c_dir = os.path.join(base_dir, dataset,'derivatives','fmriprep')
    sub_dirs = [a for a in glob.glob(os.path.join(c_dir,'sub-*')) if os.path.isdir(a)]
    n_sub = len(sub_dirs)
    print(f'{dataset}')
    for i, sub in enumerate(sub_dirs):
        sub_ID = sub.split(os.sep)[-1]
        tasks = glob.glob(os.path.join(sub, "func", "*.tsv"))
        for task in tasks: 
            task_name = task.split(os.sep)[-1].split('_')[1].split('-')[-1]
            conf_data = pd.read_csv(task, delimiter='\t')
            sub_motion.append({'dataset':dataset, 'ID':sub_ID, 
                               'task':task_name, 
                               'FD':np.mean(conf_data.framewise_displacement), 
                               'DVARS':np.mean(conf_data.std_dvars)})
        print(f'Processing... {i+1}/{n_sub}\r', end='', flush=True)
        print('\n\n')

print('\nDone!!!')

In [ ]:
sub_TIV = []
fs_atlas = 'Destrieux2009' # DesikanKilliany or Destrieux2009

# Take mean framewise displacement and DVARS from each confound file.
for dataset in datasets: 
    c_dir = os.path.join(base_dir, dataset,'derivatives','fs_stats')
    sub_dirs = [a for a in glob.glob(os.path.join(c_dir,'sub-*')) if os.path.isdir(a)]
    n_sub = len(sub_dirs)
    print(f'{dataset}')
    for i, sub in enumerate(sub_dirs):
        sub_ID = sub.split(os.sep)[-1]
        conf_data_name = os.path.join(sub, f'{sub_ID}_desc-{fs_atlas}_stats.tsv')
        conf_data = pd.read_csv(conf_data_name, delimiter='\t')
        sub_TIV.append(
            {
                'dataset':dataset, 'ID':sub_ID, 
                'TIV':conf_data.iloc[-1]['volume'], 'BrainSegVolNotVent': conf_data.iloc[-2]['volume']})
        print(f'Processing... {i+1}/{n_sub}\r', end='', flush=True)
    print('\n')

print('\nDone!!!')

In [ ]:
# Convert list of dictionaries to a dataframe and save it.

# Subject Motion
sub_motion = pd.DataFrame(sub_motion)
sub_motion.to_csv(os.path.join(base_dir,'subject_motion.csv'), index=None)

# TIV
sub_TIV = pd.DataFrame(sub_TIV)
sub_TIV.to_csv(os.path.join(base_dir,'sub_TIV.csv'), index=None)